In [10]:
# See description above for what these variables represent
big_loop_iterations = 10
explainer_runs = 10
thresholds = [30, 50]
samples = [10]

# Used to label the resulting files -
date = "23_May"

# Set up directory for result files
import os
dir = f'./results_{date}'
if not os.path.exists(dir):
    os.mkdir(dir)

In [11]:
from GNNSubNet import GNNSubNet as gnn
import pandas as pd
import numpy as np

# # Kidney data set  ------------------------- #
loc   = "../TCGA"
ppi   = f'{loc}/KIDNEY_RANDOM_PPI.txt'
feats = [f'{loc}/KIDNEY_RANDOM_Methy_FEATURES.txt', f'{loc}/KIDNEY_RANDOM_mRNA_FEATURES.txt']
targ  = f'{loc}/KIDNEY_RANDOM_TARGET.txt'

# # Synthetic data set  ------------------------- #
# loc   = "GNNSubNet/datasets/synthetic/"
# ppi   = f'{loc}/NETWORK_synthetic.txt'
# feats = [f'{loc}/FEATURES_synthetic.txt']
# targ  = f'{loc}/TARGET_synthetic.txt'

# Read in the synthetic data
g = gnn.GNNSubNet(loc, ppi, feats, targ, normalize=False)

# Get some general information about the data dimension
g.summary()

Graph is connected  False
Calculate subgraph ...
Number of subgraphs:  118
Size of subgraph:  2049
Graph is connected  True
##################
# DATASET LOADED #
##################

Number of nodes: 2049
Number of edges: 13588
Number of modalities: 2


In [18]:
def obtain_BAGEL_scores(loc, ppi, feats, targ, gnn_subnet, date):
    model_info = []
    fidelity = []
    validity_plus = []
    validity_minus = []
    validity_plus_matrix = []
    validity_minus_matrix = []
    sparsity = []

    for i in range(8, big_loop_iterations):
        g = gnn.GNNSubNet(loc, ppi, feats, targ, normalize=False)
        g.train()

        # Check the performance of the classifier
        accuracy = g.accuracy

        # Run the explainer the desired number of times
        g.explain(explainer_runs, gnn_subnet=gnn_subnet)

        # Fidelity with varying samples using varying hardmask thresholds
        # for t in thresholds:
        #     for s in samples:
        #         f = g.evaluate_RDT_fidelity_soft(samples=s, threshold=t)
        #
        #         # Save raw results in case needed for further analysis
        #         filename = f"results_{date}/{i}_fidelities_{s}_samples_top_{t}.csv"
        #         np.savetxt(filename, f, delimiter=',', fmt ='% s')
        #
        #         # Save mean fidelity to list to create processed table
        #         fidelity.append( [i, accuracy, s, t, np.mean(f)])

        # Fidelity
        f = g.evaluate_RDT_fidelity_soft()
        fidelity = ([i, accuracy, np.mean(f)])

        filename = f"results_{date}/{i}_fidelities.csv"
        np.savetxt(filename, fidelity, delimiter=',', fmt ='% s')

        # Sparsity
        sparsities = g.evaluate_sparsity()
        # Save raw results in case needed for further analysis
        filename = f"results_{date}/{i}_sparsities.csv"
        np.savetxt(filename, sparsities, delimiter=',', fmt ='% s')
        # Save mean sparsity to list to create processed table
        sparsity.append([i, accuracy, np.mean(sparsities)])

        # Validity at varying thresholds
        for t in thresholds:
            v_plus, v_minus, mat_plus, mat_minus = g.evaluate_validity(threshold=t, confusion_matrix=True)
            validity_plus.append([i, accuracy, t, v_plus])
            validity_minus.append([i, accuracy, t, v_minus])
            validity_plus_matrix.append([i, accuracy, t, mat_plus[0,0], mat_plus[0,1], mat_plus[1,0], mat_plus[1,1]])
            validity_minus_matrix.append([i, accuracy, t, mat_minus[0,0], mat_minus[0,1], mat_minus[1,0], mat_minus[1,1]])

        filename = f"results_{date}/{i}_validity_plus.csv"
        np.savetxt(filename, validity_plus_matrix, delimiter=',', fmt ='% s')

        filename = f"results_{date}/{i}_validity_minus.csv"
        np.savetxt(filename, validity_minus_matrix, delimiter=',', fmt ='% s')

In [ ]:
obtain_BAGEL_scores(loc, ppi, feats, targ, False, "modified_alg")

Graph is connected  False
Calculate subgraph ...
Number of subgraphs:  118
Size of subgraph:  2049
Graph is connected  True
##################
# DATASET LOADED #
##################
graphcnn for training ...
Graphs class 0: 200, Graphs class 1: 306
Length of balanced dataset list: 400
Train graph class 0: 158, train graph class 1: 162
Validation graph class 0: 42, validation graph class 1: 38


100%|██████████| 35/35 [00:13<00:00,  2.68batch/s]


Epoch 0, loss 3932.8112
Train Acc 0.5062
Epoch 0, val_loss 249.6641
Saving best model with validation loss 249.6641082763672


100%|██████████| 35/35 [00:11<00:00,  3.02batch/s]


Epoch 1, loss 1216.8085
Train Acc 0.5062
Epoch 1, val_loss 354.6093


...

Epoch 13, loss 19.8239
Train Acc 0.7406
Epoch 13, val_loss 7.6912


100%|██████████| 35/35 [00:13<00:00,  2.66batch/s]


Epoch 14, loss 12.2401
Train Acc 0.5094
Epoch 14, val_loss 20.9951


100%|██████████| 35/35 [00:12<00:00,  2.90batch/s]


Epoch 15, loss 11.8558
Train Acc 0.4938
Epoch 15, val_loss 26.2038
Early stopping!

Confusion matrix (Validation set):

[[26 16]
 [ 2 36]]
Validation accuracy: 77.5%
Validation loss 4.257605075836182

------- Run the Explainer -------

GNN-SubNet: False
Explainer::Iteration 1 of 10
Explainer::Iteration 2 of 10
Explainer::Iteration 3 of 10
Explainer::Iteration 4 of 10
Explainer::Iteration 5 of 10


In [ ]:
obtain_BAGEL_scores(loc, ppi, feats, targ, True, "gnn_subnet")